<a href="https://colab.research.google.com/github/abar-1/SDR-ML-Project/blob/fix-modulation-function/QAMReceiverV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Function to convert binary data to QAM16 constellations

In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


16QAM Constellation Modulator

In [2]:
import numpy as np
"""
Used https://dsplog.com/2008/06/01/binary-to-gray-code-for-16qam/ for mappings. When tested, the function works and
the binary is correctly mapped to its correspondingcomplex number based on the constellation. To check, uncomment
the last line in the cell which prints a test run of the function.
"""


def modulator(binary_data, M):
    k = int(np.log2(M))

    # defining the real and imaginary PAM constellation for 16-QAM
    alphaRe = np.arange(-(2*np.sqrt(M)/2-1), (2*np.sqrt(M)/2), 2)
    alphaIm = np.arange(-(2*np.sqrt(M)/2-1), (2*np.sqrt(M)/2), 2)

    # taking b0b1 for real
    ipDecRe = np.array([int(''.join(map(str, b[:k//2])), 2) for b in binary_data])
    ipGrayDecRe = ipDecRe ^ (ipDecRe >> 1)

    # taking b2b3 for imaginary
    ipDecIm = np.array([int(''.join(map(str, b[k//2:])), 2) for b in binary_data])
    ipGrayDecIm = ipDecIm ^ (ipDecIm >> 1)

    # mapping the Gray coded symbols into constellation
    modRe = alphaRe[ipGrayDecRe]
    modIm = alphaIm[ipGrayDecIm]

    # complex constellation
    mod = modRe + 1j * modIm

    return mod

def generate_qam_symbols(M=16, num_symbols=1000):
    # 4 bits per symbol
    k = int(np.log2(M))

    # Generate random binary data
    random_bits = np.random.randint(0, 2, num_symbols * k)

    # Reshape to match modulator input
    binary_data = random_bits.reshape((-1, k))

    # Modulate Binary Data
    modulated_symbols = modulator(binary_data, M)

    return modulated_symbols, binary_data
#print(generate_qam_symbols(16,5))

Adding noise to 16QAM signals

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def add_awgn(qam_symbols, snr_db):
    """
    Adds Additive White Gaussian Noise (AWGN) to QAM symbols.

    Parameters:
        qam_symbols (numpy array): The transmitted QAM symbols (complex numbers).
        snr_db (float): Signal-to-noise ratio in dB.

    Returns:
        numpy array: Noisy QAM symbols.
    """
    # Calculate signal power
    signal_power = np.mean(np.abs(qam_symbols) ** 2)

    # Compute noise power based on SNR (convert dB to linear scale)
    noise_power = signal_power / (10 ** (snr_db / 10))

    # Generate AWGN noise
    noise = np.sqrt(noise_power / 2) * (np.random.randn(*qam_symbols.shape) + 1j * np.random.randn(*qam_symbols.shape))

    # Add noise to the symbols
    noisy_qam_symbols = qam_symbols + noise
    return noisy_qam_symbols


# Generate 16QAM symbols
M = 16
num_symbols = 1000
qam_symbols, original_binary = generate_qam_symbols(M, num_symbols)

#Signal to noise ratio in dB
#High signal to noise ratio is good, low is bad
snr_db = 15


# Add noise to QAM symbols
noisy_qam = add_awgn(qam_symbols, snr_db)

df = pd.DataFrame({'features':original_binary.tolist(), 'target':noisy_qam.tolist()})
df.reset_index(inplace=True)

df.drop(['index'],axis=1,inplace=True)

#Getting data of different noise levels (data augmentation to prevent overfitting)
for i in range(15, 55, 5):
  M = 16
  num_symbols = 2000
  qam_symbols, original_binary = generate_qam_symbols(M, num_symbols)
  snr_db = i
  noisy_qam = add_awgn(qam_symbols, snr_db)
  df1 = pd.DataFrame({'features':original_binary.tolist(), 'target':noisy_qam.tolist()})
  df1.reset_index(inplace=True)
  df1.drop(['index'],axis=1,inplace=True)
  df = pd.concat([df, df1])

  #Plot Constellations (Before and After Noise)
  # plt.figure(figsize=(10,5))
  # plt.subplot(1,2,2)
  # plt.scatter(noisy_qam.real, noisy_qam.imag, alpha=0.5, label="Noisy")
  # plt.title(f"Signal to Noise Ratio = {snr_db} dB)")
  # plt.xlabel("In-phase (I)")
  # plt.ylabel("Quadrature (Q)")
  # plt.grid()

  #plt.show()
df = df.rename(columns={'features': 'binary','target':'complex'})
#To save as CSV
df.to_csv('dataWithNoise.csv', index = False)




Build Neural Network

X = Complex

y = Binary

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

#Target variable (binary data)
y = df['binary'].values
# Convert each element of y to a NumPy array
y = np.array([np.array(yi) for yi in y])
#Reshape to 4 bits/symbol (-1 means numpy will figure out how many rows are needed)
y = y.reshape(-1, 4)
y = y.astype(int)

# Manually convert 4-bit binary to integer (0-15)
y = np.array([int("".join(str(bit) for bit in row), 2) for row in y])

# One-hot encoding for 16 classes
y = to_categorical(y, num_classes=16)


# Features (complex data)
X = df['complex'].values
X = np.array([np.array(xi) for xi in X])

# Reshape to (n_samples, 2) for real and imaginary parts
X = [[x.real, x.imag] for x in X]
X = np.array(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(2,)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    # 16 neurons for 16 different possibilities (0000 - 1111)
    Dense(16, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/20


C:\Users\anees\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


340/340 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4346 - loss: 1.8945 - val_accuracy: 0.9838 - val_loss: 0.4247
Epoch 2/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8970 - loss: 0.4420 - val_accuracy: 0.9912 - val_loss: 0.0619
Epoch 3/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9400 - loss: 0.2441 - val_accuracy: 0.9930 - val_loss: 0.0304
Epoch 4/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9619 - loss: 0.1551 - val_accuracy: 0.9926 - val_loss: 0.0233
Epoch 5/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9719 - loss: 0.1119 - val_accuracy: 0.9926 - val_loss: 0.0233
Epoch 6/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9740 - loss: 0.0977 - val_accuracy: 0.9923 - val_loss: 0.0215
Epoch 7/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9794 - loss: 0.0782 - val_accuracy: 0.9912 - val_loss: 0.0217
Epoch 8/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9779 - loss: 0.0767 - val_accuracy: 0.9937 - val_

In [68]:
import numpy as np

X_pred = np.array([[2.5, -3], [1, -3], [3, -1], [-1, 3]])  # Convert X_pred to a NumPy array

# Assuming model.predict(X_pred) gives some output
temp = model.predict(X_pred)

predictions = []

# Find the index of the max value in each prediction
for x in range(len(temp)):
    max_val = max(temp[x])
    predictions.append(np.where(temp[x] == max_val)[0][0])
print("Complex Input: ", X_pred)


binarylist = []
for num in predictions:
    binary = ""
    if num == 0:  
        binary = "0000"
    else:
        while num > 0:
            binary = str(num % 2) + binary  
            num //= 2 
    if(len(binary)) < 4:
        binary += "0"
    binarylist.append(binary)
print("Model Predictions: ", binarylist)
comlex = ["(-3, -3)","(-3,-1)","(-3, 1)","(-3, 3)","(-1, -3)","(-1, -1)","(-1, 1)","(-1, 3)","(1, -3)","(1, -1)","(1, 1)","(1, 3)","(3, -3)","(3, -1)","(3, 1","(3, 3)"]
bin = ["0000","0001","0011","0010","0100","0101","0111","0110","1100","1101","1111","1110","1000","1001","1011","1010"]

#Making df to help test predictions
compare = pd.DataFrame(columns=[comlex, bin])
compare




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step
Complex Input:  [[ 2.5 -3. ]
 [ 1.  -3. ]
 [ 3.  -1. ]
 [-1.   3. ]]
Model Predictions:  ['1000', '1100', '1001', '1100']


,"(-3, -3)","(-3,-1)","(-3, 1)","(-3, 3)","(-1, -3)","(-1, -1)","(-1, 1)","(-1, 3)","(1, -3)","(1, -1)","(1, 1)","(1, 3)","(3, -3)","(3, -1)","(3, 1","(3, 3)"
,0000,0001,0011,0010,0100,0101,0111,0110,1100,1101,1111,1110,1000,1001,1011,1010
